# Knowledge Bases for Amazon Bedrock와 연동된 Agent for Amazon Bedrock 생성 및 Action Group 연결하기

이 노트북에서는 회사 데이터를 검색하고 작업을 완료하기 위해 Knowledge Bases for Amazon Bedrock를 사용하는 Amazon Bedrock Agent를 만드는 방법을 배웁니다. 이 노트북의 사용 사례는 레스토랑의 에이전트로, 고객에게 성인용 또는 어린이용 메뉴에 대한 정보를 제공하고 테이블 예약 시스템을 담당하는 작업을 수행합니다. 고객은 예약 정보를 생성, 삭제 또는 가져올 수 있습니다. 

이 노트북을 완성하는 단계는 다음과 같습니다:

1. 필요한 라이브러리 Import
2. 데이터 세트를 Amazon S3에 업로드
3. Knowledge Base for Amazon Bedrock 생성
4. Agent for Amazon Bedrock 생성
5. Agent 테스트하기
6. 생성된 리소스 정리

<img src="./images/lab5-architecture.png" alt="Create an Agent with a Knowledge Base and an Action Group" style="height: 400px; width:950px;"/>


## 1. Import the needed libraries

In [1]:
!pip install -q opensearch-py
!pip install -q retrying

In [2]:
import os
import json
import time
import uuid
import boto3
import zipfile
import logging
import pprint
from io import BytesIO
from retrying import retry
from itertools import cycle
from botocore.exceptions import ClientError
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from aoss_utils import createEncryptionPolicy, createNetworkPolicy, createAccessPolicy, createCollection, waitForCollectionCreation
from bedrock_utils import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, delete_iam_role_and_policies

In [3]:
#Clients
s3_client = boto3.client('s3')
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
lambda_client = boto3.client('lambda')
dynamodb_client = boto3.client('dynamodb')
dynamodb_resource = boto3.resource('dynamodb')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock = boto3.client("bedrock",region_name=region)
opensearch_client = boto3.client('opensearchserverless')
account_id = sts_client.get_caller_identity()["Account"]
identity_arn = session.client('sts').get_caller_identity()['Arn']
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '322537213286')

In [5]:
suffix = f"{region}-{account_id}"
agent_name = 'booking-agent'
solution_id = agent_name
agent_alias_name = "booking-agent-alias"
bucket_name = f'{agent_name}-{suffix}'

## 2. Upload the dataset to Amazon S3
Knowledge Bases for Amazon Bedrock, currently require data to reside in an Amazon S3 bucket. In this section we will create an Amazon S3 bucket and the files.

### 2.1 Create the Amazon S3 bucket

In [6]:
if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name.lower(),
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

### 2.2 Upload dataset to the Amazon S3 bucket

In [7]:
dataset_path = "dataset"
files = [f.name for f in os.scandir(dataset_path) if f.is_file()]
for file in files:
    s3_client.upload_file(f'{dataset_path}/{file}', bucket_name, f'{file}')

## 3. Create a Knowledge Base for Amazon Bedrock

이 섹션에서는 Knowledge Base.를 만들고 테스트하는 모든 단계를 살펴봅니다. 

완료해야 할 단계는 다음과 같습니다:
    
1. Amazon Opensearch Serverless Index 생성
2. Embeddings model ARN 정의
3. Knowledge Base 생성
4. data source 생성
5. Knowledge Base 동기화
6. Knowledge Base 테스트

### 3.1 Amazon Opensearch Serverless Service Index 생성

Amazon Opensearch Serverless Service에서 신규 인덱스를 생성하려면 다음 단계를 완료해야 합니다:
- Amazon Bedrock execution role 생성
- OpenSearch Service에서 정책을 생성하고 collection을 생성
- 인덱스 설정 및 매핑을 정의하고 인덱스를 생성

In [8]:
def short_uuid():
    uuid_str = str(uuid.uuid4())
    return uuid_str[:8]

short_uuid = short_uuid()

collectionName = "{}-{}".format(solution_id, short_uuid)
indexName = "{}-index-{}".format(solution_id, short_uuid)

print("Collection name:",collectionName)
print("Index name:",indexName)

Collection name: booking-agent-06deab41
Index name: booking-agent-index-06deab41


#### Amazon Bedrock execution role 생성

In [9]:
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

#### OpenSearch Service 정책 생성과 collection 생성

In [10]:
# Create policies in OpenSearch Service (OSS)
encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=collectionName,
    aoss_client=opensearch_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn
)

# Create collection in OpenSearch Service
collection = opensearch_client.create_collection(
    name=collectionName,
    type='VECTORSEARCH'
)

# Wait for collection creation to complete
time.sleep(10)

# Extract collection ID and host
collection_detail = collection.get('createCollectionDetail', {})
collection_id = collection_detail.get('id', '')
host = f"{collection_id}.{region}.aoss.amazonaws.com"

# Print collection details and host
print("Collection:", collection)
print("Host:", host)

# Create OSS policy and attach it to Bedrock execution role
create_oss_policy_attach_bedrock_execution_role(
    collection_id=collection_id,
    bedrock_kb_execution_role=bedrock_kb_execution_role
)
# Wait for all elements to be created
time.sleep(40) 

Collection: {'createCollectionDetail': {'arn': 'arn:aws:aoss:us-west-2:322537213286:collection/pq16ymgfmelm9zd381t0', 'createdDate': 1717816491185, 'id': 'pq16ymgfmelm9zd381t0', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1717816491185, 'name': 'booking-agent-06deab41', 'standbyReplicas': 'ENABLED', 'status': 'CREATING', 'type': 'VECTORSEARCH'}, 'ResponseMetadata': {'RequestId': 'aa1e98fc-7089-4e96-8683-c00aac18f670', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'aa1e98fc-7089-4e96-8683-c00aac18f670', 'date': 'Sat, 08 Jun 2024 03:14:51 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '314', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
Host: pq16ymgfmelm9zd381t0.us-west-2.aoss.amazonaws.com
Opensearch serverless arn:  arn:aws:iam::322537213286:policy/AmazonBedrockOSSPolicyForKnowledgeBase_475


#### index settings and mappings 정의와 index 생성

In [11]:
# Set up AWS authentication
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region, service)

# Define index settings and mappings
index_settings = {
    "settings": {
        "index.knn": "true"
    },
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1536,
                 "method": {
                     "name": "hnsw",
                     "engine": "faiss",
                     "space_type": "innerproduct",
                     "parameters": {
                         "ef_construction": 512,
                         "m": 16
                     },
                 },
             },
            "text": {
                "type": "text"
            },
            "text-metadata": {
                "type": "text"
            }
        }
    }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# Create index
response = oss_client.indices.create(index=indexName,body=json.dumps(index_settings))
print(response)

[2024-06-08 03:15:41,622] p4608 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
[2024-06-08 03:15:42,666] p4608 {base.py:258} INFO - PUT https://pq16ymgfmelm9zd381t0.us-west-2.aoss.amazonaws.com:443/booking-agent-index-06deab41 [status:200 request:1.043s]


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'booking-agent-index-06deab41'}


### 3.2 Embedding Model ARN 정의
Knowledge Bases for Amazon Bedrock에 데이터를 색인하는 데 사용할 임베딩 모델 ARN을 정의합니다.

In [12]:
embeddingModelArn = "arn:aws:bedrock:{}::foundation-model/amazon.titan-embed-text-v1".format(region)

### 3.3 Knowledge Base 생성
이 섹션에서는 이전에 생성한 Amazon Bedrock 실행 역할, 임베딩 모델 ARN 및 Opensearch 구성을 제공하는 Knowledge Base를 생성합니다. 

In [14]:
knowledge_base_name = solution_id
description = "Agents for Amazon Bedrock와 결합할 수 있는 Restaurant KB 에시"
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": indexName,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

In [15]:
@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = knowledge_base_name,
        description = description,
        roleArn = bedrock_kb_execution_role_arn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

try:
    kb = create_knowledge_base_func()
    kb_id = kb["knowledgeBaseId"]
    print(kb)
except Exception as err:
    print(f"{err=}, {type(err)=}")

{'createdAt': datetime.datetime(2024, 6, 8, 3, 18, 51, 883799, tzinfo=tzlocal()), 'description': 'Agents for Amazon Bedrock와 결합할 수 있는 Restaurant KB 에시', 'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:322537213286:knowledge-base/OZHOFK82QI', 'knowledgeBaseConfiguration': {'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1'}}, 'knowledgeBaseId': 'OZHOFK82QI', 'name': 'booking-agent', 'roleArn': 'arn:aws:iam::322537213286:role/AmazonBedrockExecutionRoleForKnowledgeBase_475', 'status': 'CREATING', 'storageConfiguration': {'opensearchServerlessConfiguration': {'collectionArn': 'arn:aws:aoss:us-west-2:322537213286:collection/pq16ymgfmelm9zd381t0', 'fieldMapping': {'metadataField': 'text-metadata', 'textField': 'text', 'vectorField': 'vector'}, 'vectorIndexName': 'booking-agent-index-06deab41'}, 'type': 'OPENSEARCH_SERVERLESS'}, 'updatedAt': datetime.datetime(2024, 6, 8, 3, 18, 51, 883799, tzinfo=

### 3.4 Create the data source (S3) 생성

knowledge base 생성한 후, 데이터 소스를 knowledge base에 수집하여 색인화하여 쿼리할 수 있도록 합니다.

In [16]:
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}"
}

data_source = bedrock_agent_client.create_data_source(
    knowledgeBaseId=kb_id,
    name='restaurant-menus-source',
    description='Location of the restaurant menus',
    dataSourceConfiguration={
        'type': 'S3',
        's3Configuration': s3Configuration
    },
    vectorIngestionConfiguration={
        'chunkingConfiguration': chunkingStrategyConfiguration
    }
)

data_source_id = data_source["dataSource"]["dataSourceId"]
print("The data source id is: ", data_source_id)

The data source id is:  BOHBWOHX5G


### 3.5 Knowledge Base 동기화
데이터 소스를 만들고 Knowledge Base에 연결했으므로 데이터 동기화를 진행할 수 있습니다. 


데이터 소스에 대한 S3 버킷에서 파일을 추가, 수정 또는 제거할 때마다 데이터 소스를 동기화하여 Knowledge Base에 다시 색인되도록 해야 합니다. 동기화는 증분 방식으로 이루어지므로 Amazon Bedrock은 마지막 동기화 이후 추가, 수정 또는 삭제된 S3 버킷의 개체만 처리합니다.

In [17]:
ingestion_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb_id,
    dataSourceId=data_source_id,
    description='Initial Ingestion'
)

In [18]:
%%time
status = bedrock_agent_client.get_ingestion_job(
    knowledgeBaseId=ingestion_job_response["ingestionJob"]["knowledgeBaseId"],
    dataSourceId=ingestion_job_response["ingestionJob"]["dataSourceId"],
    ingestionJobId=ingestion_job_response["ingestionJob"]["ingestionJobId"]
)["ingestionJob"]["status"]
print(status)
while status not in ["COMPLETE", "FAILED", "STOPPED"]:
    status = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=ingestion_job_response["ingestionJob"]["knowledgeBaseId"],
        dataSourceId=ingestion_job_response["ingestionJob"]["dataSourceId"],
        ingestionJobId=ingestion_job_response["ingestionJob"]["ingestionJobId"]
    )["ingestionJob"]["status"]
    print(status)
    time.sleep(30)

IN_PROGRESS
IN_PROGRESS
COMPLETE
CPU times: user 7.86 ms, sys: 3.48 ms, total: 11.3 ms
Wall time: 1min


### 3.6 Knowledge Base 테스트
이제 Knowlegde Base를 사용할 수 있게 되었으니 **retrieve** 및 **retrieve_and_생성** 함수를 사용하여 테스트해 볼 수 있습니다. 

In [19]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which are the 5 mains available in the childrens menu?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0".format(region),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The 5 mains available in the children's menu are:

1. Mini Cheeseburgers - Small beef patties topped with cheese, served on mini buns.
2. Fish Sticks - Breaded fish sticks served with tartar sauce.
3. Grilled Cheese Sandwich - Melted cheese between slices of buttered bread, grilled to perfection.
4. Spaghetti with Marinara Sauce - Kid-friendly spaghetti noodles topped with tomato marinara sauce.
5. Mini Pita Pizza - Small pita bread topped with tomato sauce, cheese, and favorite toppings.



In [23]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
                } 
            },
    retrievalQuery={
        'text': '어린이 메뉴로 이용할 수 있는 5가지 메인 메뉴는 무엇인가요?'
            
        }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents
# each list has content,location,score,metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Chunk 1:  The Regrettable Experience — Children's Menu Entrees:   1. CHICKEN NUGGETS   ●   ●   ●   Description: Crispy chicken nuggets served with a side of ketchup or ranch dressing.   Allergens: Gluten (in the coating), possible Soy.   Suitable for Vegetarians: No   2. MACARONI AND CHEESE   ●   ●   ●   Description: Classic macaroni pasta smothered in creamy cheese sauce.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   3. MINI CHEESE QUESADILLAS   ●   ●   ●   Description: Small flour tortillas filled with melted cheese, served with a mild salsa.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   4. PEANUT BUTTER AND BANANA SANDWICH   ●   ●   ●   Description: Peanut butter and banana slices on whole wheat bread.   Allergens: Nuts (peanut), Gluten.   Suitable for Vegetarians: Yes (if using vegetarian peanut butter)   5. VEGGIE PITA POCKETS   ●   ●   ●   Description: Mini whole wheat pita pockets filled with hummus, cucumber, and cherry tomatoes.   Allergens: Glu

## 4. Agent for Amazon Bedrock 생성

이 섹션에서는 Agent for Amazon Bedrock를 생성하는 모든 단계를 살펴봅니다. 

완료해야 할 단계는 다음과 같습니다:
    
1. Amazon DynamoDB 테이블 생성
2. AWS Lambda function 생성
3. 에이전트에 필요한 IAM 정책 생성
4. Agent 생성
5. Agent Action Group 만들기
6. Agent가 Action Group Lambda를 호출하도록 허용
7. Knowledge Base를 agent에게 연결
8. Agent 준비 및 alias 생성

### 4.1 Create the DynamoDB table 생성
레스토랑 예약 정보가 포함된 DynamoDB 테이블을 생성합니다.

In [21]:
table_name = 'restaurant_bookings'
table = dynamodb_resource.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'booking_id',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'booking_id',
            'AttributeType': 'S'
        }
    ],
    BillingMode='PAY_PER_REQUEST'  # Use on-demand capacity mode
)

# Wait for the table to be created
print(f'Creating table {table_name}...')
table.wait_until_exists()
print(f'Table {table_name} created successfully!')

Creating table restaurant_bookings...
Table restaurant_bookings created successfully!


### 4.2 Lambda Function 생성

이제 DynamoDB 테이블과 상호 작용하는 람다 함수를 만들어 보겠습니다. 이를 위해 다음과 같이 하겠습니다:

1. 람다 함수에 대한 로직이 포함된 `lambda_function.py` 파일을 생성합니다.
2. 람다 함수에 대한 IAM 역할을 생성합니다.
3. 필요한 권한으로 람다 함수를 생성합니다.

#### Create the function code

In [24]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_bookings')

def get_booking_details(booking_id):
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'{booking_id} ID로 예약을 찾을 수 없습니다.'}
    except Exception as e:
        return {'error': str(e)}

def create_booking(date, hour, num_guests):
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        return {'error': str(e)}

def delete_booking(booking_id):
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'{booking_id} ID로 예약을 삭제하지 못했습니다.'}
    except Exception as e:
        return {'error': str(e)}

def lambda_handler(event, context):
    actionGroup = event.get('actionGroup', '')
    function = event.get('function', '')
    parameters = event.get('parameters', [])

    if function == 'get_booking_details':
        booking_id = None
        for param in parameters:
            if param["name"] == "booking_id":
                booking_id = param["value"]

        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    elif function == 'create_booking':
        date = None
        hour = None
        num_guests = None
        for param in parameters:
            if param["name"] == "date":
                date = param["value"]
            if param["name"] == "hour":
                hour = param["value"]
            if param["name"] == "num_guests":
                num_guests = int(param["value"])

        if date and hour and num_guests:
            response = str(create_booking(date, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'delete_booking':
        booking_id = None
        for param in parameters:
            if param["name"] == "booking_id":
                booking_id = param["value"]

        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Overwriting lambda_function.py


#### Create the required permissions

In [25]:
lambda_function_role = f'{solution_id}-lambda-role'

In [26]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

# Attach the AWSLambdaBasicExecutionRole policy
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

# Create a policy to grant access to the DynamoDB table
dynamodb_access_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:DeleteItem"
            ],
            "Resource": "arn:aws:dynamodb:{}:{}:table/{}".format(region, account_id, table_name)
        }
    ]
}

# Create the policy
dynamodb_access_policy_json = json.dumps(dynamodb_access_policy)
dynamodb_access_policy_response = iam_client.create_policy(
    PolicyName='{}-DynamoDBAccessPolicy'.format(solution_id),
    PolicyDocument=dynamodb_access_policy_json
)

# Attach the policy to the Lambda function's role
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn=dynamodb_access_policy_response['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '15beda15-a3a1-42bb-a50c-1e249a77a44b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:29:54 GMT',
   'x-amzn-requestid': '15beda15-a3a1-42bb-a50c-1e249a77a44b',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Create the function 

In [27]:
lambda_function_name = f'{solution_id}-lambda'

In [28]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=60,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

### 4.3 Agent에게 필요한 IAM 정책 생성
먼저 bedrock 모델 호출 및 Knowledge Base 쿼리를 허용하는 agent 정책과 이와 관련된 정책으로 agent IAM 역할을 만들어야 합니다. 이 agent이 Claude Sonnet 모델을 호출할 수 있도록 허용합니다.

In [34]:
agent_bedrock_allow_policy_name = f"{solution_id}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solution_id}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

In [35]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        },
        {
            "Sid": "QueryKB",
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve",
                "bedrock:RetrieveAndGenerate"
            ],
            "Resource": [
                f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
            ]
        },
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [36]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'd125a713-0c90-42b0-b44f-1fd1673f7170',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:32:48 GMT',
   'x-amzn-requestid': 'd125a713-0c90-42b0-b44f-1fd1673f7170',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### 4.4 Agent 생성
필요한 IAM 역할이 만들어지면 기반 Agent 클라이언트를 사용하여 신규 Agent를 만들 수 있습니다. 이를 위해 `create_agent` 함수를 사용합니다. 여기에는 Agent 이름, 밑줄 foundation 모델 및 instruction이 필요합니다. Agent 설명을 제공할 수도 있습니다. 생성된 Agent은 아직 준비되지 않은 상태입니다. Agent를 준비한 다음 이를 사용하여 작업을 호출하고 다른 API를 사용하는 데 중점을 두겠습니다.

In [38]:
agent_description = "레스토랑 테이블 예약을 담당하는 Agent"
agent_instruction = """
너는 고객이 예약에서 정보를 검색할 수 있도록 돕는 레스토랑 agent입니다, 
신규 예약을 생성하거나 기존 예약을 삭제합니다.
"""

In [39]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '1ee8f530-c920-4e01-ba6d-d5a96abe5a3d',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:36:02 GMT',
   'content-type': 'application/json',
   'content-length': '700',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1ee8f530-c920-4e01-ba6d-d5a96abe5a3d',
   'x-amz-apigw-id': 'ZB2xVGk-vHcEJ2w=',
   'x-amzn-trace-id': 'Root=1-6663d1a1-322f12ad4da70af110c8414b'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:322537213286:agent/2QKJMRAFUN',
  'agentId': '2QKJMRAFUN',
  'agentName': 'booking-agent',
  'agentResourceRoleArn': 'arn:aws:iam::322537213286:role/AmazonBedrockExecutionRoleForAgents_booking-agent',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 6, 8, 3, 36, 2, 27041, tzinfo=tzlocal()),
  'description': '레스토랑 테이블 예약을 담당하는 Agent',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'instruction': '\n너는 고객이 예약에서 정보를 검색할 수 있도록

In [40]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

The agent id is: 2QKJMRAFUN


### 4.5 Agent Action Group 생성
이제 앞에서 만든 람다 함수를 사용하는 agent action group을 만들어 보겠습니다. 이 기능은 [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) 함수가 제공합니다. 아직 agent 버전이나 alias을 만들지 않았으므로 `DRAFT`를 agent 버전으로 사용하겠습니다. agent에게 action group 기능을 알리기 위해 action group의 기능이 포함된 action group 설명을 제공합니다.

이 예에서는 `functionSchema`를 사용하여 action group 기능을 제공합니다.

함수 스키마를 사용하여 함수를 정의하려면 각 함수에 대한 `name`, `description`와 `parameters`를 제공해야 합니다.

In [42]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': '레스토랑 예약 세부 정보 검색',
        'parameters': {
            "booking_id": {
                "description": "검색할 예약 ID",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': '신규 레스토랑 예약 생성',
        'parameters': {
            "date": {
                "description": "예약 일자",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "예약 시간",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "예약 게스트 수",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': '기존 레스토랑 예약 삭제',
        'parameters': {
            "booking_id": {
                "description": "삭제할 예약 ID",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [43]:
agent_action_group_description = """
테이블 예약 정보 가져와서 신규 예약 생성 또는 기존 예약 삭제 Actions"""

agent_action_group_name = "TableBookingsActionGroup"

In [44]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [45]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '0db0d360-9f3a-49e9-9e27-9b29855f7e69',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:42:06 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0db0d360-9f3a-49e9-9e27-9b29855f7e69',
   'x-amz-apigw-id': 'ZB3qWGWEvHcEp1w=',
   'x-amzn-trace-id': 'Root=1-6663d30e-29d6d7d3400577030ce873c3'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:322537213286:function:booking-agent-lambda'},
  'actionGroupId': 'V0U6HABCLL',
  'actionGroupName': 'TableBookingsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': '2QKJMRAFUN',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 6, 8, 3, 42, 6, 864358, tzinfo=tzlocal()),
  'description': '\n테이블 예약 정보 가져와서 신규 예약 생성 또는 기존 예약 삭제 Actions',
  'functionSchema': {'functions': [{'description': '레스토랑 예약 세부 정보 검색',
     'name': 'get_booking_detail

### 4.6 Agent가 Action Group Lambda를 호출하도록 허용
action group을 사용하기 전에 agent이 action group과 연결된 람다 함수를 호출할 수 있도록 허용해야 합니다. 이는 리소스 기반 정책을 통해 이루어집니다. 생성된 람다 함수에 리소스 기반 정책을 추가해 보겠습니다.

In [46]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [47]:
response

{'ResponseMetadata': {'RequestId': '384a7d6b-f82c-481a-824d-873a434bdeb8',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:44:56 GMT',
   'content-type': 'application/json',
   'content-length': '348',
   'connection': 'keep-alive',
   'x-amzn-requestid': '384a7d6b-f82c-481a-824d-873a434bdeb8'},
  'RetryAttempts': 1},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:322537213286:function:booking-agent-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:322537213286:agent/2QKJMRAFUN"}}}'}

### 4.7 Knowledge Base를 agent에게 연계
이제 Agent를 만들었으므로 앞서 만든 Knowledge Base를 연결할 수 있습니다. 

In [48]:
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='고객이 메뉴에 대해 문의할 때 knowledge base에 접근',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

In [49]:
response

{'ResponseMetadata': {'RequestId': 'a7a22abb-2723-46eb-95eb-a39b95d020a8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:48:39 GMT',
   'content-type': 'application/json',
   'content-length': '258',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a7a22abb-2723-46eb-95eb-a39b95d020a8',
   'x-amz-apigw-id': 'ZB4nxGoyvHcEsgA=',
   'x-amzn-trace-id': 'Root=1-6663d497-30bc7980323ba3b947e23463'},
  'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2024, 6, 8, 3, 48, 39, 927053, tzinfo=tzlocal()),
  'description': '고객이 메뉴에 대해 문의할 때 knowledge base에 접근',
  'knowledgeBaseId': 'OZHOFK82QI',
  'knowledgeBaseState': 'ENABLED',
  'updatedAt': datetime.datetime(2024, 6, 8, 3, 48, 39, 927053, tzinfo=tzlocal())}}

### 4.8 Agent 준비와 alias 생성

내부 테스트 동안 사용할 수 있는 agent의 DRAFT 버전을 생성합니다.

In [50]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': 'c4b9ec79-7db0-4d41-a571-b5665dbbeebb', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 08 Jun 2024 03:50:04 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'c4b9ec79-7db0-4d41-a571-b5665dbbeebb', 'x-amz-apigw-id': 'ZB41AEuXPHcERzg=', 'x-amzn-trace-id': 'Root=1-6663d4ec-4352482c3c3a991939e36edf'}, 'RetryAttempts': 0}, 'agentId': '2QKJMRAFUN', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 6, 8, 3, 50, 4, 693796, tzinfo=tzlocal())}


We are also going to create an Agent alias to later on use to invoke it

In [51]:
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='TestAlias',
    agentId=agent_id,
    description='Test alias',
)

alias_id = response["agentAlias"]["agentAliasId"]
print("The Agent alias is:",alias_id)

The Agent alias is: 2DYLWGXKMP


## 5. Agent 테스트
agent를 만들었으니 `bedrock-agent-runtime` 클라이언트를 사용하여 이 에이전트를 호출하고 몇 가지 작업을 수행해 보겠습니다.

In [52]:
def invokeAgent(query, session, enable_trace=False):
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [53]:
session_id:str = str(uuid.uuid1())
query = "어린이 메뉴의 시작 메뉴는 무엇인가요?"
response = invokeAgent(query, session_id)
print(response)

어린이 메뉴의 시작 메뉴는 치킨 너겟입니다. 치킨 너겟은 튀김옷을 입힌 후 튀겨낸 것으로, 케첩이나 랜치 드레싱과 함께 제공됩니다. 치킨 너겟에는 글루텐(밀가루)과 가능성 있는 대두 알레르기 성분이 포함되어 있습니다. 채식주의자에게는 적합하지 않습니다.


In [54]:
query = "5월 5일 오후 8시에 2명 예약하고 싶어요."
response = invokeAgent(query, session_id)
print(response)

예약이 완료되었습니다. 예약 ID는 2859f6d4입니다.


In [55]:
query = "2859f6d4 예약에 대한 정보를 받고 싶습니다."
response = invokeAgent(query, session_id)
print(response)

예약 ID 2859f6d4에 대한 세부 정보는 다음과 같습니다:

날짜: 2023년 5월 5일
시간: 오후 8시 
게스트 수: 2명


In [56]:
query = "1e421532 예약에 대한 정보를 알고 싶어요."
response = invokeAgent(query, session_id)
print(response)

죄송합니다. 1e421532 ID로 된 예약 내역을 찾을 수 없습니다.


In [57]:
query = "2859f6d4 예약을 취소해 주세요."
response = invokeAgent(query, session_id)
print(response)

예약 ID 2859f6d4가 성공적으로 취소되었습니다.


In [58]:
query = "2859f6d4 예약을 확인해 주세요."
response = invokeAgent(query, session_id)
print(response)

예약 ID 2859f6d4는 이미 취소된 상태입니다. 따라서 해당 예약에 대한 정보를 찾을 수 없습니다.


Now show a call with full trace

In [59]:
session_id:str = str(uuid.uuid1())
query = "성인용 메뉴에는 어떤 디저트가 있나요?"
response = invokeAgent(query, session_id, enable_trace=True)
print(response)

[2024-06-08 04:03:02,592] p4608 {3120418367.py:15} INFO - None
[2024-06-08 04:03:02,739] p4608 {3120418367.py:30} INFO - {
  "agentAliasId": "2DYLWGXKMP",
  "agentId": "2QKJMRAFUN",
  "agentVersion": "1",
  "sessionId": "0058499c-254c-11ef-9989-0a86a609e469",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\" \ub108\ub294 \uace0\uac1d\uc774 \uc608\uc57d\uc5d0\uc11c \uc815\ubcf4\ub97c \uac80\uc0c9\ud560 \uc218 \uc788\ub3c4\ub85d \ub3d5\ub294 \ub808\uc2a4\ud1a0\ub791 agent\uc785\ub2c8\ub2e4, \uc2e0\uaddc \uc608\uc57d\uc744 \uc0dd\uc131\ud558\uac70\ub098 \uae30\uc874 \uc608\uc57d\uc744 \uc0ad\uc81c\ud569\ub2c8\ub2e4. You have been provided with a set of functions to answ

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sat, 08 Jun 2024 04:03:02 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '0058499c-254c-11ef-9989-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '84e7b5d9-1a1f-4c99-9305-9b7754a9a40f'},
                      'HTTPStatusCode': 200,
                      'RequestId': '84e7b5d9-1a1f-4c99-9305-9b7754a9a40f',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f8560340bb0>,
 'contentType': 'application/json',
 'sessionId': '0058499c-254c-11ef-9989-0a86a609e469'}


[2024-06-08 04:03:06,197] p4608 {3120418367.py:30} INFO - {
  "agentAliasId": "2DYLWGXKMP",
  "agentId": "2QKJMRAFUN",
  "agentVersion": "1",
  "sessionId": "0058499c-254c-11ef-9989-0a86a609e469",
  "trace": {
    "orchestrationTrace": {
      "rationale": {
        "text": "\uace0\uac1d\uc774 \uc131\uc778\uc6a9 \uba54\ub274\uc758 \ub514\uc800\ud2b8 \uc635\uc158\uc5d0 \ub300\ud574 \ubb38\uc758\ud558\uace0 \uc788\uc2b5\ub2c8\ub2e4. \uc774\ub97c \ud655\uc778\ud558\uae30 \uc704\ud574 knowledge base\ub97c \uac80\uc0c9\ud574\uc57c \ud569\ub2c8\ub2e4.",
        "traceId": "84e7b5d9-1a1f-4c99-9305-9b7754a9a40f-0"
      }
    }
  }
}
[2024-06-08 04:03:06,198] p4608 {3120418367.py:30} INFO - {
  "agentAliasId": "2DYLWGXKMP",
  "agentId": "2QKJMRAFUN",
  "agentVersion": "1",
  "sessionId": "0058499c-254c-11ef-9989-0a86a609e469",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledg

성인용 메뉴에는 다음과 같은 디저트 옵션이 있습니다:

1. 클래식 뉴욕 치즈케이크 - 그래함 크래커 크러스트에 크리미한 치즈케이크, 과일 컴포트나 초콜릿 가나슈 토핑 제공. 알레르기 유발 물질: 유제품, 글루텐.

2. 애플 파이 아라모드 - 따뜻한 애플 파이에 바닐라 아이스크림과 카라멜 소스를 곁들여 제공. 알레르기 유발 물질: 유제품, 글루텐.

3. 초콜릿 라바 케이크 - 부드럽고 촉촉한 초콜릿 케이크에 녹아있는 초콜릿 필링, 설탕가루를 뿌리고 라즈베리 셔벗을 곁들여 제공. 알레르기 유발 물질: 유제품, 가능성 있는 글루텐, 가능성 있는 콩.

4. 피칸 파이 바 - 버터 쇼트브레드 크러스트 위에 피칸 필링을 올려 바 형태로 제공. 알레르기 유발 물질: 유제품, 계란, 글루텐.

5. 바나나 푸딩 파르페 - 바닐라 푸딩, 바나나 슬라이스, 바닐라 웨이퍼를 층층이 쌓아 올리고 휘핑크림과 캐러멜 가루를 토핑으로 제공. 알레르기 유발 물질: 유제품, 글루텐.


## 6. Clean-up 
Let's delete all the associated resources created to avoid unnecessary costs. 

In [60]:
action_group_id=agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']


In [61]:
import boto3

def clean_up_resources(
    agent_id, alias_id, action_group_id, action_group_name,
    lambda_function_name, bucket_name, kb_id, collection_id, agent_role_name, table_name
):

    # Delete Agent Action Group, Agent Alias, and Agent
    try:
        response = bedrock_agent_client.update_agent_action_group(
                        agentId=agent_id,
                        agentVersion='DRAFT',
                        actionGroupId= action_group_id,
                        actionGroupName=action_group_name,
                        actionGroupExecutor={
                            'lambda': lambda_function['FunctionArn']
                        },
                        functionSchema={
                            'functions': agent_functions
                        },
                        actionGroupState='DISABLED',
                    )
        bedrock_agent_client.delete_agent_action_group(agentId=agent_id, agentVersion='DRAFT', 
                                                       actionGroupId=action_group_id)
        bedrock_agent_client.delete_agent_alias(agentAliasId=alias_id, agentId=agent_id)
        bedrock_agent_client.delete_agent(agentId=agent_id)
        print(f"Agent {agent_id}, Agent Alias {alias_id}, and Action Group have been deleted.")
    except Exception as e:
        print(f"Error deleting Agent resources: {e}")

    # Delete Lambda function
    try:
        lambda_client.delete_function(FunctionName=lambda_function_name)
        print(f"Lambda function {lambda_function_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting Lambda function {lambda_function_name}: {e}")

    # Delete all objects in the bucket and the bucket itself
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            for obj in response['Contents']:
                s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"Bucket {bucket_name} and its objects have been deleted.")
    except Exception as e:
        print(f"Error deleting bucket {bucket_name}: {e}")

    # Delete Knowledge Base
    try:
        bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
        print(f"Knowledge Base {kb_id} has been deleted.")
    except Exception as e:
        print(f"Error deleting Knowledge Base {kb_id}: {e}")

    # Delete Opensearch Collection
    try:
        opensearch_client.delete_collection(id=collection_id)
        print(f"Collection {collection_id} has been deleted.")
    except Exception as e:
        print(f"Error deleting Collection {collection_id}: {e}")

    # Delete Lambda Role and its policies
    try:
        attached_policies = iam_client.list_attached_role_policies(RoleName=agent_role_name)['AttachedPolicies']
        for policy in attached_policies:
            policy_name = policy['PolicyName']
            iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=policy['PolicyArn'])
            iam_client.delete_role_policy(RoleName=agent_role_name, PolicyName=policy_name)
            print(f"Detached and deleted policy {policy_name} from role {agent_role_name}")
        iam_client.delete_role(RoleName=agent_role_name)
        print(f"Role {agent_role_name} has been deleted.")
    except Exception as e:
        pass
        print(f"Error deleting role {agent_role_name}: {e}")

    # Delete DynamoDB table
    try:
        dynamodb_client.delete_table(TableName=table_name)
        print(f"Table {table_name} is being deleted...")
        waiter = dynamodb_client.get_waiter('table_not_exists')
        waiter.wait(TableName=table_name)
        print(f"Table {table_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting table {table_name}: {e}")

In [62]:
clean_up_resources(agent_id, alias_id, 
                   action_group_id, action_group_name, 
                   lambda_function_name, bucket_name, 
                   kb_id, collection_id, agent_role_name, table_name)
delete_iam_role_and_policies()

Agent 2QKJMRAFUN, Agent Alias 2DYLWGXKMP, and Action Group have been deleted.
Lambda function booking-agent-lambda has been deleted.
Bucket booking-agent-us-west-2-322537213286 and its objects have been deleted.
Knowledge Base OZHOFK82QI has been deleted.
Collection pq16ymgfmelm9zd381t0 has been deleted.
Error deleting role AmazonBedrockExecutionRoleForAgents_booking-agent: An error occurred (NoSuchEntity) when calling the DeleteRolePolicy operation: The role policy with name booking-agent-ba cannot be found.
Table restaurant_bookings is being deleted...
Table restaurant_bookings has been deleted.


0